In [ ]:
import os
import os.path as op

import matplotlib.image as mpimg
from neuromaps import transforms
from matplotlib.gridspec import GridSpec
import pandas as pd
from gradec.utils import _zero_medial_wall
import nibabel as nib
from nilearn.plotting.cm import _cmap_d as nilearn_cmaps
from surfplot import Plot
from nilearn import datasets
from gradec.plot import plot_radar, plot_cloud
from gradec.fetcher import _fetch_features, _fetch_frequencies, _fetch_classification
from gradec.utils import _decoding_filter
from nimare.decode.continuous import CorrelationDecoder
import numpy as np
from nilearn.maskers import NiftiMasker
from nilearn.image import threshold_img, index_img
from neuromaps.datasets import fetch_fslr
from nimare.transforms import p_to_z
from nilearn.plotting import plot_stat_map
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import nilearn.reporting

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from nilearn import datasets, plotting, image

In [ ]:
CMAP = nilearn_cmaps["cold_hot"]
CMAP = "Spectral_r"

In [ ]:
# Define the number of colors in the colormap
num_colors = 256

# Define anchor points for key colors emphasizing red and blue
colors = [
    (0.0, 0.0, 0.2),  # Purple
    (0.0, 0.0, 0.3),  # Purple
    (0.0, 0.0, 0.6),  # Purple
    (0.1, 0.0, 0.9),  # Cyan
    (0.1, 0.3, 0.9),  # Cyan
    (0.1, 0.8, 0.9),  # Cyan
    (0.9, 0.7, 0.3),  # Yellow (adjusted)
    (1.0, 0.5, 0.2),  # Orange
    (1.0, 0.4, 0.2),  # Orange
    (1.0, 0.0, 0.3),  # Red
    (1.0, 0.0, 0.3),  # Red
]  #
# Create a smooth interpolation for the custom colormap
CMAP = LinearSegmentedColormap.from_list("custom_cmap", colors, N=num_colors)

plt.figure(figsize=(8, 2))
plt.imshow([list(range(256))], aspect="auto", cmap=CMAP)
plt.gca().set_visible(False)
plt.colorbar(orientation="horizontal")
plt.show()

In [ ]:
# Define the number of colors in the colormap
num_colors = 256

# Define anchor points for key colors emphasizing red and blue
colors = [
    (0.0, 0.0, 0.2),  # Purple
    (0.0, 0.0, 0.6),  # Purple
    (0.2, 0.4, 0.9),  # Cyan
    (0.9, 0.7, 0.3),  # Yellow (adjusted)
    (1.0, 0.5, 0.2),  # Orange
    (1.0, 0.4, 0.2),  # Orange
    (1.0, 0.0, 0.3),  # Red
    (1.0, 0.0, 0.3),  # Red
]  #
# Create a smooth interpolation for the custom colormap
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=num_colors)

plt.figure(figsize=(8, 2))
plt.imshow([list(range(256))], aspect="auto", cmap=cmap)
plt.gca().set_visible(False)
plt.colorbar(orientation="horizontal")
plt.show()

In [ ]:
# Define the number of colors in the colormap
num_colors = 256

# Define anchor points for key colors emphasizing red and blue
colors = [
    (0.0, 0.0, 0.2),  # Purple
    (0.1, 0.0, 0.9),  # Cyan
    (0.1, 0.3, 0.9),  # Cyan
    (0.1, 0.8, 0.9),  # Cyan
    (0.9, 0.7, 0.4),  # Yellow (adjusted)
    (0.9, 0.7, 0.3),  # Yellow (adjusted)
    (1.0, 0.5, 0.2),  # Orange
    (1.0, 0.4, 0.2),  # Orange
    (1.0, 0.0, 0.3),  # Red
    (1.0, 0.0, 0.3),  # Red
]  #
# Create a smooth interpolation for the custom colormap
CMAP2 = LinearSegmentedColormap.from_list("custom_cmap", colors, N=num_colors)

plt.figure(figsize=(8, 2))
plt.imshow([list(range(256))], aspect="auto", cmap=CMAP2)
plt.gca().set_visible(False)
plt.colorbar(orientation="horizontal")
plt.show()

In [ ]:
def trim_image(img=None, tol=1, fix=True):
    if fix:
        mask = img != tol
    else:
        mask = img <= tol
    if img.ndim == 3:
        mask = mask.any(2)
    mask0, mask1 = mask.any(0), mask.any(1)
    return img[np.ix_(mask1, mask0)]

In [ ]:
def plot_stat(nii_img_thr, threshold, mask_contours=None, vmax=8, alpha=1, cmap=CMAP):
    # Fetch the MNI152 template at higher resolution
    template = datasets.load_mni152_template(resolution=0.5)

    fig = plt.figure()

    plotting.plot_stat_map(
        nii_img_thr,
        bg_img=template,
        black_bg=False,
        draw_cross=False,
        annotate=True,
        alpha=alpha,
        cmap=cmap,
        threshold=threshold,
        cut_coords=[1, 1, -28],
        vmax=vmax
    )

    return fig

In [ ]:
def plot_vol(nii_img_thr, threshold, mask_contours=None, vmax=6, alpha=1, cmap=CMAP):
    template = datasets.load_mni152_template(resolution=1)

    display_modes = ["x", "y", "z"]
    fig = plt.figure(figsize=(5, 5))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
    gs = GridSpec(2, 2, figure=fig)

    for dsp_i, display_mode in enumerate(display_modes):
        if display_mode == "z":
            ax = fig.add_subplot(gs[:, 1], aspect="equal")
            colorbar = True
        else:
            ax = fig.add_subplot(gs[dsp_i, 0], aspect="equal")
            colorbar = False

        display = plot_stat_map(
            nii_img_thr,
            bg_img=template,
            black_bg=False,
            draw_cross=False,
            annotate=True,
            alpha=alpha,
            cmap=cmap,
            threshold=threshold,
            colorbar=colorbar,
            display_mode=display_mode,
            cut_coords=1,
            vmax=vmax,
            axes=ax,
        )
        if mask_contours:
            display.add_contours(mask_contours, levels=[0.5], colors="black")

    return fig

In [ ]:
def plot_surf(nii_img_thr, mask_contours=None, vmax=8, cmap=cmap):
    map_lh, map_rh = transforms.mni152_to_fslr(nii_img_thr, fslr_density="32k")
    map_lh, map_rh = _zero_medial_wall(
        map_lh,
        map_rh,
        space="fsLR",
        density="32k",
    )
    # midthickness

    surfaces = fetch_fslr(density="32k")
    lh, rh = surfaces["inflated"]
    sulc_lh, sulc_rh = surfaces["sulc"]

    p = Plot(surf_lh=lh, surf_rh=rh, layout="grid")
    p.add_layer({"left": sulc_lh, "right": sulc_rh}, cmap="binary_r", cbar=False)
    p.add_layer(
        {"left": map_lh, "right": map_rh},
        cmap=cmap,
        cbar=False,
        color_range=(-vmax, vmax)
    )
    if mask_contours:
        mask_lh, mask_rh = transforms.mni152_to_fslr(mask_contours, fslr_density="32k")
        mask_lh, mask_rh = _zero_medial_wall(
            mask_lh,
            mask_rh,
            space="fsLR",
            density="32k",
        )
        mask_arr_lh = mask_lh.agg_data()
        mask_arr_rh = mask_rh.agg_data()
        countours_lh = np.zeros_like(mask_arr_lh)
        countours_lh[mask_arr_lh != 0] = 1
        countours_rh = np.zeros_like(mask_arr_rh)
        countours_rh[mask_arr_rh != 0] = 1

        colors = [(0, 0, 0, 0)]
        contour_cmap = ListedColormap(colors, 'regions', N=1)
        line_cmap = ListedColormap(["black"], 'regions', N=1)
        p.add_layer(
            {"left": countours_lh, "right": countours_rh}, 
            cmap=line_cmap, 
            as_outline=True, 
            cbar=False
        )
        p.add_layer(
            {"left": countours_lh, "right": countours_rh},
            cmap=contour_cmap,
            cbar=False,
        )
    
    return p.build()

In [ ]:
MED = False
data_dir = op.join("./dset")
abide_dir = op.join(data_dir, "group-medication/habenula") if MED else op.join(data_dir, "group/habenula")
fig_dir = op.join(abide_dir, "fig")
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

In [ ]:
DSET, MODEL = "neuroquery", "lda"
decoding_dir = "./decoding"
decoder_fn = op.join(decoding_dir, f"{MODEL}_{DSET}_decoder.pkl.gz")

decoder = CorrelationDecoder.load(decoder_fn)

features = _fetch_features(DSET, MODEL, data_dir=decoding_dir)
frequencies = _fetch_frequencies(DSET, MODEL, data_dir=decoding_dir)
classification, class_lst = _fetch_classification(DSET, MODEL, data_dir=decoding_dir)

In [ ]:
mask_img = datasets.load_mni152_brain_mask(resolution=1)
masker = NiftiMasker(mask_img=mask_img)
masker = masker.fit()

In [ ]:
test_dict = {
    "1s": "Group Average (One-Sample T-Test)",
    "2s": "Group Comparison (Two-Sample Unpaired T-Test: ASD-TD)",
}

In [ ]:
brik_fn = op.join(abide_dir, "sub-group_task-rest_desc-1S2StTesthabenula_briks+tlrc.BRIK")
table_fn = op.join(abide_dir, "sub-group_task-rest_desc-1S2StTesthabenula_table.txt")
nii_1s_fn = op.join(abide_dir, "sub-group_task-rest_desc-1SampletTest_zmap.nii.gz")
nii_2s_fn = op.join(abide_dir, "sub-group_task-rest_desc-2SampletTest_zmap.nii.gz")
cluster_fn = op.join(data_dir, "group-medication", "habenula", "clustsim", "clustsim_out.NN2_2sided.1D")

column_names = [".10000", ".05000", ".02000", ".01000"]
cluster_df = pd.read_table(cluster_fn, skiprows=8, delim_whitespace=True, names=column_names)
cluster_df = cluster_df.reset_index()
cluster_df.rename(columns={'index': 'pthr'}, inplace=True)


brik_idx = [9, 11] if MED else [8, 10]
nii_fns = [nii_1s_fn, nii_2s_fn]
tests = ["1s", "2s"]
alpha = ".02000"
pthrs = [0.0001, 0.0001]
cohen_thresh = 0

data_df = pd.read_csv(table_fn, sep="\t")
n_sub = data_df.groupby("group").size().sum()
n_sub_1, n_sub_2 = data_df.groupby("group").size().values

for brik_i, nii_fn, test, pthr in zip(brik_idx, nii_fns, tests, pthrs):
    convert = f"3dAFNItoNIFTI \
        -prefix {nii_fn} \
        {brik_fn}[{brik_i}]"
    print(convert)
    os.system(convert)

    '''if test == "1s":
        convert = f"3dAFNItoNIFTI \
            -prefix {nii_fn} \
            {brik_fn}[{brik_i}]"
        os.system(convert)

    elif test == "2s":
        convert = f"3dcalc -a {brik_fn}'[{brik_i}]' -expr '-1*a' -prefix {nii_fn}"
        os.system(convert)'''

    nii_img = nib.load(nii_fn)
    z_thresh = p_to_z(pthr)

    clust_ext = cluster_df.loc[cluster_df["pthr"] == pthr, alpha].values[0]
    nii_thr_img = threshold_img(nii_img, z_thresh, cluster_threshold=clust_ext)
    print(clust_ext, pthr, z_thresh)

    nii_arr = masker.transform(nii_img)
    if test == "1s":
        nii_cohen_arr = nii_arr / np.sqrt(n_sub)
    elif test == "2s":
        nii_cohen_arr = nii_arr / (np.sqrt(n_sub_1) + np.sqrt(n_sub_2))

    nii_cohen_img = masker.inverse_transform(nii_cohen_arr)

    nii_thr_arr = masker.transform(nii_thr_img)
    nii_contour_arr = np.zeros_like(nii_thr_arr)
    nii_contour_arr[(nii_thr_arr > z_thresh) | (nii_thr_arr < -z_thresh)] = 1
    nii_contour_img = masker.inverse_transform(nii_contour_arr)
    nii_contour_img_3d = index_img(nii_contour_img, 0)

    vmax = round(np.max(np.abs(nii_thr_arr)), 2)
    vmax = 13 if vmax > 13 else vmax
    # c_vmax = round(np.max(np.abs(nii_cohen_arr)), 2)
    c_vmax = 0.1 if test == "2s" else 0.3

    clusters = nilearn.reporting.get_clusters_table(
        nii_thr_img, z_thresh, two_sided=True
    )
    print(clusters)  # coordinates are same as affine of input (MNI)

    nib.save(nii_thr_img, op.join(abide_dir, f"{test}_thresh_zmap.nii.gz"))

    # stat_fig = plot_vol(nii_thr_img, z_thresh, vmax=vmax)
    vol_fig = plot_vol(nii_thr_img, z_thresh, vmax=vmax)
    cohen_fig = plot_vol(nii_cohen_img, cohen_thresh, mask_contours=nii_contour_img_3d, vmax=c_vmax, alpha=0.8, cmap=CMAP)

    surf_fig = plot_surf(nii_thr_img, vmax=vmax)
    cohen_surf_fig = plot_surf(nii_cohen_img, mask_contours=nii_contour_img_3d, vmax=c_vmax, cmap=CMAP)

    if test == "1s":
        nii_pos_arr = np.where(nii_arr > 0, nii_arr, 0)
        img_to_decode = masker.inverse_transform(nii_pos_arr)
    elif test == "2s":
        nii_neg_arr = abs(np.where(nii_arr < 0, nii_arr, 0))
        img_to_decode = masker.inverse_transform(nii_neg_arr)

    corrs_df = decoder.transform(img_to_decode)
    num_val = [int(lab.split("__")[1].split("_")[0]) for lab in corrs_df.index.to_list()]
    indices = np.argsort(num_val)
    corrs_df = corrs_df.iloc[indices]
    filtered_df, filtered_features, filtered_frequencies = _decoding_filter(
        corrs_df,
        features,
        classification,
        freq_by_topic=frequencies,
        class_by_topic=class_lst,
    )

    # Visualize results
    corrs = filtered_df["r"].to_numpy()
    # Radar plot
    '''plot_radar(
        corrs, 
        filtered_features,
        MODEL,
        cmap=CMAP2,
        out_fig=op.join(fig_dir, f"{test}_radar.png"),
    )

    # Word cloud plot
    plot_cloud(
        corrs, 
        filtered_features,
        MODEL,
        width=5,
        height=10,
        frequencies=filtered_frequencies,
        cmap=CMAP2,
        out_fig=op.join(fig_dir, f"{test}_wordcloud.png"),
    )'''
    # plt.show(stat_fig)

    '''stat_fig.savefig(
        op.join(fig_dir, f"{test}_stat.png"), bbox_inches="tight", dpi=300
    )'''
    # vol_fig.savefig(op.join(fig_dir, f"{test}_volume.png"), bbox_inches="tight", dpi=300)
    # cohen_fig.savefig(op.join(fig_dir, f"{test}_volume-cohen.png"), bbox_inches="tight", dpi=300)
    # surf_fig.savefig(op.join(fig_dir, f"{test}_surface.png"), bbox_inches="tight", dpi=300)
    # cohen_surf_fig.savefig(op.join(fig_dir, f"{test}_surface-cohen.png"), bbox_inches="tight", dpi=300)

In [ ]:
import os
import subprocess
import pandas as pd
import nibabel as nib
import numpy as np
from nilearn.image import threshold_img
from nilearn.reporting import get_clusters_table


# Function to execute the AFNI whereami command and parse its output
def find_region(coord):
    # Format the coordinates properly for the whereami command
    coord_str = f"{coord[0]} {coord[1]} {coord[2]}"
    # Command to query AFNI's whereami
    whereami_cmd = f"whereami [{coord_str}] -atlas MNI_Glasser_HCP_v1.0 -classic"
    try:
        result = subprocess.run(
            whereami_cmd,
            shell=True,
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
        )
        output = result.stdout
        # Print the full output for debugging purposes
        print(f"whereami output for {coord}:")
        print(output)
        # Parse the output to extract the region name
        for line in output.split("\n"):
            if "Focus point (LPI)" in line:
                parts = line.split()
                if len(parts) > 3:
                    return parts[3]  # Extract the region name
        return "Unknown"
    except subprocess.CalledProcessError as e:
        print(
            f"Error running whereami command: {e}\nOutput: {e.output}\nError: {e.stderr}"
        )
        return "Unknown"


# Extract the coordinates from the clusters table and map to regions
def map_clusters_to_regions(clusters_df):
    coordinates = clusters_df[["X", "Y", "Z"]].values
    regions = [find_region(coord) for coord in coordinates]
    clusters_df["Region"] = regions
    return clusters_df


# Iterate over your clusters and map to regions
for brik_i, nii_fn, test, pthr in zip(brik_idx, nii_fns, tests, pthrs):
    convert = f"3dAFNItoNIFTI -prefix {nii_fn} {brik_fn}[{brik_i}]"
    os.system(convert)

    nii_img = nib.load(nii_fn)
    z_thresh = p_to_z(pthr)
    info = nii_img.get_fdata()

    clust_ext = cluster_df.loc[cluster_df["pthr"] == pthr, alpha].values[0]
    nii_thr_img = threshold_img(nii_img, z_thresh, cluster_threshold=clust_ext)

    clusters = get_clusters_table(nii_thr_img, stat_threshold=z_thresh, two_sided=True)
    clusters_with_regions = map_clusters_to_regions(clusters)


In [ ]:
from nilearn import datasets
from nilearn.image import load_img
from nilearn.input_data import NiftiLabelsMasker
import numpy as np

# Load the Juelich atlas
juelich_atlas = datasets.fetch_atlas_juelich("maxprob-thr0-1mm")
atlas_filename = juelich_atlas.maps
atlas_labels = juelich_atlas.labels

# Load the Harvard-Oxford atlas
harvard_oxford_atlas = datasets.fetch_atlas_harvard_oxford("sub-maxprob-thr25-1mm")
atlas_filename = harvard_oxford_atlas.maps
atlas_labels = harvard_oxford_atlas.labels

# Load the atlas image
atlas_img = load_img(atlas_filename)


# Function to find the region for a given coordinate
def find_region(coord):
    # Transform MNI coordinates to voxel indices
    voxel_indices = np.round(
        nib.affines.apply_affine(np.linalg.inv(atlas_img.affine), coord)
    ).astype(int)
    # Get the label index for the voxel
    label_idx = atlas_img.get_fdata()[tuple(voxel_indices)]
    region_name = (
        atlas_labels[int(label_idx)]
        if int(label_idx) < len(atlas_labels)
        else "Unknown"
    )
    return region_name


# Extract the coordinates from the clusters table and map to regions
def map_clusters_to_regions(clusters_df):
    coordinates = clusters_df[["X", "Y", "Z"]].values
    regions = [find_region(coord) for coord in coordinates]
    clusters_df["Region"] = regions
    return clusters_df


# Iterate over your clusters and map to regions
for brik_i, nii_fn, test, pthr in zip(brik_idx, nii_fns, tests, pthrs):
    convert = f"3dAFNItoNIFTI -prefix {nii_fn} {brik_fn}[{brik_i}]"
    os.system(convert)

    nii_img = nib.load(nii_fn)
    z_thresh = p_to_z(pthr)
    info = nii_img.get_fdata()

    clust_ext = cluster_df.loc[cluster_df["pthr"] == pthr, alpha].values[0]
    nii_thr_img = threshold_img(nii_img, z_thresh, cluster_threshold=clust_ext)
    print(clust_ext, pthr, z_thresh)

    clusters = nilearn.reporting.get_clusters_table(
        nii_thr_img, stat_threshold=z_thresh, two_sided=True
    )
    clusters_with_regions = map_clusters_to_regions(clusters)
    print(clusters_with_regions)

    output_filename = op.join(abide_dir, f"clusters_with_subregions_{test}_pthr_{pthr}.csv")
    clusters_with_regions.to_csv(output_filename, index=False)
    print(f"Saved clusters with regions to {output_filename}")




## Combine sub-figures

In [ ]:
for test in tests:
    output_file = op.join(fig_dir, f"0_{test}.png")
    
    figure1 = plt.figure(figsize=(10, 6))
    gs = GridSpec(nrows=2, ncols=3, figure=figure1)

    for col, data_type in enumerate(["", "-cohen"]):
        img_files = [ 
            op.join(fig_dir, f"{test}_volume{data_type}.png"),
            op.join(fig_dir, f"{test}_surface{data_type}.png"),
        ]

        for row, fn in enumerate(img_files):
            img1 = mpimg.imread(fn)
            #if row == 0:
            #    img1 = trim_image(img=img1, tol=1, fix=True)

            ax = figure1.add_subplot(gs[row, col], aspect="equal")
            ax.imshow(img1)

            if col == 0 and row == 0:
                ax.set_title("Z-Map", fontsize=12)
            if col == 1 and row == 0:
                ax.set_title("Effect Size", fontsize=12)
            ax.set_axis_off()

    dec_fn = op.join(fig_dir, f"{test}_wordcloud.png")
    img1 = mpimg.imread(dec_fn)
    ax = figure1.add_subplot(gs[0:2, 2], aspect="equal")
    ax.imshow(img1)
    ax.set_axis_off()

    figure1.suptitle(test_dict[test], fontsize=14)
    figure1.subplots_adjust(wspace=0.1, hspace=0.1) # top=1.2
    figure1.savefig(output_file, bbox_inches="tight", dpi=300)
    plt.show()

## QC of connectivity data

In [ ]:
import ast

summart_stats = pd.read_csv(op.join("./abide", "rsfc-qc.csv"))
summart_stats["Score"] = summart_stats["Score"].apply(lambda x: ast.literal_eval(x)[0] if isinstance(x, str) and x.startswith('[') else x)
summart_stats["Score"] = summart_stats["Score"].astype(float)

In [ ]:
import plotly.express as px

fig = px.strip(
    summart_stats,
    y="Score",
    color="ID",
    facet_col="Stat",
    stripmode="group",
    facet_col_wrap=4,
    facet_col_spacing=0.08,
)

fig.update_xaxes(showline=True, linewidth=2, linecolor="black", mirror=True)
fig.update_yaxes(
    constrain="domain",
    matches=None,
    showline=True,
    linewidth=2,
    linecolor="black",
    mirror=True,
    title=None,
)
fig.update_layout(
    height=900,
    autosize=True,
    font_size=14,
    plot_bgcolor="white",
    xaxis_gridcolor="white",
    yaxis_gridcolor="white",
    xaxis_gridwidth=2,
    showlegend=False,
)
fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
fig.write_html(op.join(fig_dir, "summary_stats.html"), full_html=True, include_plotlyjs=True)

## Get outliers from participants.tsv

In [ ]:
ppt_df = pd.read_csv("/Users/jperaza/Desktop/participants.tsv", sep="\t")

rater_columns = [
    "qc_rater_1", 
    "qc_anat_rater_2", 
    "qc_func_rater_2",
    "qc_anat_rater_3",
    "qc_func_rater_3",
]

ppt_to_exclude = []
for col in rater_columns:
    ppt_to_exclude.extend(ppt_df[ppt_df[col] == "fail"]["participant_id"].to_list())

pd.DataFrame(list(set(ppt_to_exclude))).to_csv(op.join(fig_dir, "exclude_ppts.csv"), index=False, header=False)